In [ ]:
# default_exp modules

In [ ]:
#export

from abc import ABCMeta
from dataclasses import dataclass, field
from traitlets import HasTraits, Dict
from abc import ABCMeta, abstractmethod
from io import StringIO
from typing import Optional, Mapping, Any, Iterable
import traittypes
from IPython.display import display
from ipywidgets import FileUpload, Output, Widget
import pandas as pd

@dataclass
class DisplayItem():
    name: str
    display_object: Widget
    metadata: Mapping[str, Any] = field(default_factory=dict)

    def __repr__(self):

        return f"{self.__class__.__name__}(name={self.name} display_type={self.display_object.__class__.__name__} metadata={self.metadata})"


class DharpaModule(metaclass=ABCMeta):

    def __init__(self, **config):

        self._config: Mapping[str, Any] = config
        self._values: HasTraits = self._get_values()
        self._display_items: Iterable[DisplayItem] = self._get_display_items()

    @property
    def values(self):
        return self._values

    def test(self):
        pass

    @property
    def display_items(self):
        return self._display_items

    @abstractmethod
    def _get_values(self) -> HasTraits:
        pass

    @abstractmethod
    def _get_display_items(self) -> Iterable[DisplayItem]:
        pass

    def __repr__(self):

        return f"{self.__class__.__name__}(value_names={self.values.trait_names()} display_items={self.display_items})"


class FileInputModule(DharpaModule):

    def _get_values(self) -> HasTraits:

        class InputFileValue(HasTraits):

            file_details = Dict()
            content_data = traittypes.DataFrame()

        return InputFileValue()

    def _get_display_items(self) -> Iterable[DisplayItem]:

        uploaded_file_details = Output()
        file_upload = FileUpload(accept=".csv", multiple=False)

        def upload_handler(change):

            uploaded_file = list(change.new.values())[0]
            data = uploaded_file["content"].decode("utf-8")
            df = pd.read_csv(StringIO(data), header='infer')

            uploaded_file_details.clear_output()
            with uploaded_file_details:
                print(uploaded_file["metadata"])
                display(df)

        file_upload.observe(upload_handler, names=['value'])

        return [DisplayItem(name="file_details", display_object=uploaded_file_details),
            DisplayItem(name="file_upload", display_object=file_upload)]


In [ ]:
from dharpa_toolbox.core import export_notebooks
export_notebooks()


Converted 00_core.ipynb.
Converted 01_modules.ipynb.
Converted 02_workflows.ipynb.
